# HNSW
HNSW is one of the best working ANN algorithm

In [ ]:
import numpy as np

# now define a function to read the fvecs file format of Sift1M dataset
def read_fvecs(fp):
    a = np.fromfile(fp, dtype='int32')
    d = a[0]
    return a.reshape(-1, d + 1)[:, 1:].copy().view('float32')

# data we will search through
wb = read_fvecs('./sift/sift_base.fvecs')  # 1M samples
# also get some query vectors to search with
xq = read_fvecs('./sift/sift_query.fvecs')
# take just one query (there are many in sift_learn.fvecs)
xq = xq[0].reshape(1, xq.shape[1])

wb.shape, xq.shape

((1000000, 128), (1, 128))

In [4]:
import faiss
d = wb.shape[1]  # dimension
M = 32 # number of Neighbours
index = faiss.IndexHNSWFlat(d, M)

In [5]:
# Nothing is created in the HNSW index
index.hnsw.max_level

-1

In [8]:
levels = faiss.vector_to_array(index.hnsw.levels)
np.bincount(levels)

array([], dtype=int64)

In [ ]:
index.add(wb)
# This might be longer than usual as HNSW builds the graph structure

In [10]:
index.hnsw.max_level

4

In [35]:
levels = faiss.vector_to_array(index.hnsw.levels)
np.bincount(levels)

array([     0, 968746,  30276,    951,     26,      1], dtype=int64)

``` array([     0, 968746,  30276,    951,     26,      1], dtype=int64)```
```
968746 - layer 0
30276  - layer 1
951    - layer 2
26     - layer 3
1      - Entry Point

```

In [12]:
# We can check the vector that is being used as entry points
index.hnsw.entry_point

118295

In [14]:
# lets just look at the probabilty function used to assign levels to nodes, from OFFICIAL FAISS GITHUB REPO
# THe official is in cpp, we convert to python here for understanding. CPP is more efficient.
def set_default_probas(M: int, m_L: float):
    nn = 0  # set nearest neighbors count = 0
    cum_nneighbor_per_level = []
    level = 0  # we start at level 0
    assign_probas = []
    while True:
        # calculate probability for current level
        proba = np.exp(-level / m_L) * (1 - np.exp(-1 / m_L))
        # once we reach low prob threshold, we've created enough levels
        if proba < 1e-9: break
        assign_probas.append(proba)
        # neighbors is == M on every level except level 0 where == M*2
        nn += M*2 if level == 0 else M
        cum_nneighbor_per_level.append(nn)
        level += 1
    return assign_probas, cum_nneighbor_per_level

Here we are building two vectors — assign_probas, the probability of insertion at a given layer, and cum_nneighbor_per_level, the cumulative total of nearest neighbors assigned to a vertex at different insertion levels

In [15]:
# lets see how many levels are created for M = 32 manually
assign_probas, cum_nneighbor_per_level = set_default_probas(32, 1/np.log(32))
assign_probas, cum_nneighbor_per_level

([0.96875,
  0.030273437499999986,
  0.0009460449218749991,
  2.956390380859371e-05,
  9.23871994018553e-07,
  2.887099981307982e-08],
 [64, 96, 128, 160, 192, 224])

From this, we can see the significantly higher probability of inserting a vector at level 0 than higher levels (although, as we will explain below, the probability is not exactly as defined here). 

Our assign_probas vector is used by another method called random_level — it is in this function that each vertex is assigned an insertion level.

We generate a random float using Numpy’s random number generator rng (initialized below) in f. For each level, we check if f is less than the assigned probability for that level in assign_probas — if so, that is our insertion layer.

If f is too high, we subtract the assign_probas value from f and try again for the next level. The result of this logic is that vectors are most likely going to be inserted at level 0. Still, if not, there is a decreasing probability of insertion at ease increment level.

Finally, if no levels satisfy the probability condition, we insert the vector at the highest level with return len(assign_probas) - 1

In [ ]:

def random_level(assign_probas, rng):
    r = rng.uniform() # random number between 0 and 1

    for level in range(len(assign_probas)):
        if r < assign_probas[level]:
            return level
        
        r = r - assign_probas[level]

    return len(assign_probas) - 1

In [32]:
rng = np.random.default_rng(seed=12345)
manual_levels = [random_level(assign_probas, rng) for _ in range(wb.shape[0])]

In [33]:
np.bincount(manual_levels)

array([968821,  30170,    985,     23,      1], dtype=int64)

In [36]:
np.bincount(levels)

array([     0, 968746,  30276,    951,     26,      1], dtype=int64)

If you compare the manually generated levels and HNSW levels they are almost the same. its actually cool I think

Now that we’ve explored all there is to explore on the theory behind HNSW and how this is implemented in Faiss — let’s look at the effect of different parameters on our recall, search and build times, and the memory usage of each.

Our **efConstruction** value must be set before we construct the index via index.add(xb), but **efSearch** can be set anytime before searching.



In [38]:

efConstruction = 32
efSearch = 16
M = 32

### After trying different M, efConstruction and efSearch values, we can set them as follows:
- ef construction affects the index building time more than search time
- ef search affects the search time more than index building time
- M controls the number of neighbors, higher M means better accuracy but more memory and slower search
-  We can also increase efConstruction to achieve higher recall at lower M and efSearch values.

In [41]:
index.hnsw.efConstruction = efConstruction
index.add(wb)  # build the index
index.hnsw.efSearch = efSearch


In [48]:
%%timeit
index.search(xq, 5)  # search 5 nearest neighbors

1.2 ms ± 19.3 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
